In [3]:
# Install Dash and its dependencies
!pip install dash dash-core-components dash-html-components dash-table pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [4]:
# Dash App - Análisis de Clientes
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

In [5]:
# cargamos los datos
df = pd.read_csv("clientes.csv")

In [7]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Segmentación de Clientes - App con Dash"),

    html.Label("Seleccionar tipo de cliente:"),
    dcc.Dropdown(
        options=[{"label": tipo, "value": tipo} for tipo in df["tipo_cliente"].unique()],
        value=df["tipo_cliente"].unique().tolist(),
        multi=True,
        id="tipo-dropdown"
    ),

    html.Label("Rango de ingresos mensuales (ARS):"),
    dcc.RangeSlider(
        min=df["ingresos_mensuales"].min(),
        max=df["ingresos_mensuales"].max(),
        step=5000,
        marks=None,
        value=[df["ingresos_mensuales"].min(), df["ingresos_mensuales"].max()],
        id="ingresos-slider"
    ),

    html.Div(id="kpi-output"),

    dcc.Graph(id="scatter-plot")
])

@app.callback(
    [Output("scatter-plot", "figure"),
     Output("kpi-output", "children")],
    [Input("tipo-dropdown", "value"),
     Input("ingresos-slider", "value")]
)
def update_output(tipos, ingresos_range):
    filtered_df = df[(df["tipo_cliente"].isin(tipos)) & (df["ingresos_mensuales"].between(*ingresos_range))]
    fig = px.scatter(
        filtered_df,
        x="ingresos_mensuales",
        y="gasto_promedio_mensual",
        color="tipo_cliente",
        title="Ingresos vs Gasto mensual"
    )

    edad_avg = round(filtered_df["edad"].mean(), 1)
    ingreso_avg = int(filtered_df["ingresos_mensuales"].mean())
    gasto_avg = int(filtered_df["gasto_promedio_mensual"].mean())
    total = filtered_df.shape[0]

    kpis = html.Div([
        html.H4(f"Total de clientes filtrados: {total}"),
        html.P(f"Edad promedio: {edad_avg} años"),
        html.P(f"Ingreso medio: ${ingreso_avg:,}"),
        html.P(f"Gasto promedio: ${gasto_avg:,}")
    ])
    return fig, kpis

if __name__ == "__main__":
    app.run(debug=True)


<IPython.core.display.Javascript object>